<a href="https://colab.research.google.com/github/musthafavakkayil/Mack/blob/main/Trainig_Testing_and_deplyoing_MACK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TRAINING THE MODEL**



In [2]:
#install keras and tensorflow packages
#import the required packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator

In [3]:
#creating the required model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))

In [4]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [6]:
#training and validating the dataset
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
val_datagen = ImageDataGenerator(rescale = 1./255)
#training
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/BFH-Mack project/DataSet/Train',
                                                 target_size = (64, 64),
                                                 batch_size = 8,
                                                 class_mode = 'binary')
#validating
val_set = val_datagen.flow_from_directory('/content/drive/MyDrive/BFH-Mack project/DataSet/Val',
                                            target_size = (64, 64),
                                            batch_size = 8,
                                            class_mode = 'binary')


Found 120 images belonging to 2 classes.
Found 111 images belonging to 2 classes.


In [ ]:
#training the model and obtaining accuracy
model.fit_generator(training_set,
                         steps_per_epoch = 10,
                         epochs = 50,
                         validation_data = val_set,
                         validation_steps = 2)


In [8]:
import tensorflow as tf 

In [9]:
#saving the trained model
tf.keras.models.save_model(model,'my_model.hdf5')

**Testing the trained model**

In [10]:
import numpy as np
from keras.preprocessing import image


In [11]:
model=tf.keras.models.load_model('/content/my_model.hdf5')

In [12]:
#function to give the output for each prediction
def classify(img_file):
    img_name = img_file
    test_image = image.load_img(img_name, target_size = (64, 64))

    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = model.predict(test_image)

    if result[0][0] == 1:
        prediction = 'Mango'
    else:
        prediction = 'Jackfruit'
    print(prediction,img_name)


In [13]:
#each file in the test dataset is checked
import os
path = '/content/drive/MyDrive/BFH-Mack project/DataSet/Test'
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
   for file in f:
     #can change the file type to jpg,png,jpeg as required
     if '.jpg' in file:
       files.append(os.path.join(r, file))

In [ ]:
#printing prediction for each file
for f in files:
   classify(f)
   print('\n')

**DEPLOYING USING STREAMLIT AND NGROK**

In [ ]:
#instal streamlit and pyngrok
!pip install streamlit

In [ ]:
!pip install pyngrok

In [ ]:
#Web app 
%%writefile app.py
import streamlit as st
import tensorflow as tf

st.set_option('deprecation.showfileUploaderEncoding',False)
@st.cache(allow_output_mutation=True)
def load_model():
  model=tf.keras.models.load_model('/content/my_model.hdf5')
  return model
model=load_model()
st.write("""
            Mango or Jackfruit Classifier
         """
        )
file=st.file_uploader("Please upload an image of mango or jackfruit",type=["jpg"])
#import cv2
from PIL import Image,ImageOps
import numpy as np
def import_and_predict(image_data,model):

  size=(64,64)
  image= ImageOps.fit(image_data,size,Image.ANTIALIAS)
  img=np.asarray(image)
  img_reshape=img[np.newaxis,...]
  prediction=model.predict(img_reshape)
  if prediction[0][0]==1:
    preds='Mango'
  else:
    preds='Jackfruit'
  return preds  
  return prediction


if file is None:
  st.text("Please upload an image file")
else:
  image=Image.open(file)
  st.image(image,use_column_width=True)
  predictions = import_and_predict(image,model)
  #class_names=['Mango','Jackfruit']
  string="This image is most likely:"+predictions
  st.success(string)


In [ ]:
!ls

In [ ]:
#enter your ngrok authentication code 
!ngrok authtoken XXXXXX

In [41]:
#import ngrok from pyngrok
from pyngrok import ngrok

In [45]:
!streamlit run --server.port 80 app.py&>/dev/null&

In [ ]:
!pgrep streamlit

In [ ]:
#obtaining public url
url=ngrok.connect(port='8501')

In [ ]:
url